In [92]:
#Classificadores Lineares
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression


#Classificadores KNN
from sklearn.neighbors import KNeighborsClassifier

#Classificadores Naive Nayes
from sklearn.naive_bayes import GaussianNB

#Classificadores Arvores de Decisão
from sklearn.tree import DecisionTreeClassifier

#SVM
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, classification_report

from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import math

import scipy.stats as ss

O Teorema de Bayes fornece uma maneira de calcular a probabilidade de um dado pertencente a uma determinada classe,
considerando conhecimento prévio. O teorema de Bayes é afirmado como:


    P(class|data) = (P(data|class) * P(class)) / P(data)

Naive Bayes é um algoritmo de classificação para problemas de classificação binária (duas classes) e multiclasses. 

Chama-se Naive Bayes porque os cálculos das probabilidades para cada classe são simplificados 
para tornar seus cálculos tratáveis.

Em vez de tentar calcular as probabilidades de cada valor de atributo, 
elas são assumidas como independentes condicionalmente, considerando o valor da classe.

Essa é uma suposição muito forte que é mais improvável em dados reais, ou seja, que os atributos não interagem. 
No entanto, a abordagem tem um desempenho surpreendentemente bom em dados em que essa suposição não se aplica.



### Base de teste com 10 linhas

In [115]:
test = pd.read_csv('test.csv')
print(test.describe())
print(test['class'].unique())
print(test)

            col1       col2      class
count  10.000000  10.000000  10.000000
mean    5.178333   2.998468   0.500000
std     2.766585   1.218556   0.527046
min     1.343809   0.791637   0.000000
25%     3.180938   2.376230   0.000000
50%     4.663673   3.117676   0.500000
75%     7.700447   3.506515   1.000000
max     9.172169   4.696523   1.000000
[0 1]
       col1      col2  class
0  3.393533  2.331273      0
1  3.110073  1.781540      0
2  1.343809  3.368361      0
3  3.582294  4.679179      0
4  2.280362  2.866990      0
5  7.423437  4.696523      1
6  5.745052  3.533990      1
7  9.172169  2.511101      1
8  7.792783  3.424089      1
9  7.939821  0.791637      1


### separando os dados entre classe 0 e 1
### A partir do conjunto de dados de cada classe, obter média e desvio padrão de cada coluna

In [117]:
testClass0 = test.loc[test['class']==0]
print(testClass0.describe())

c0col1M = testClass0['col1'].mean()
c0col1std =testClass0['col1'].std()
c0col2M = testClass0['col2'].mean()
c0col2std =testClass0['col2'].std()
print("class 0" , c0col1M, c0col1std, c0col2M, c0col2std)

testClass1 = test.loc[test['class']==1]
print(testClass1.describe())

c1col1M = testClass1['col1'].mean()
c1col1std =testClass1['col1'].std()
c1col2M = testClass1['col2'].mean()
c1col2std =testClass1['col2'].std()

print("class 1" , c1col1M, c1col1std, c1col2M, c1col2std)

           col1      col2  class
count  5.000000  5.000000    5.0
mean   2.742014  3.005469    0.0
std    0.926568  1.107330    0.0
min    1.343809  1.781540    0.0
25%    2.280362  2.331273    0.0
50%    3.110073  2.866990    0.0
75%    3.393533  3.368361    0.0
max    3.582294  4.679179    0.0
class 0 2.7420144012 0.9265683289298019 3.0054686692 1.1073295894898725
           col1      col2  class
count  5.000000  5.000000    5.0
mean   7.614652  2.991468    1.0
std    1.234432  1.454193    0.0
min    5.745052  0.791637    1.0
25%    7.423437  2.511101    1.0
50%    7.792783  3.424089    1.0
75%    7.939821  3.533990    1.0
max    9.172169  4.696523    1.0
class 1 7.6146523718 1.2344321550313706 2.9914679790000003 1.454193138460162


### Usando scipy para obter probabilidade de uma distribuição normal usando média e desvio padrão

In [95]:
#Testando probabilidade normal
ss.norm(c0col1M,c0col1std).pdf(3.393533)

0.3362559728230224

### Função para cálculo de probabilidade de uma linha da base fazer parte da classe 0 e 1

In [122]:
def prob(col1,col2):
    # obtendo probabilidade acumulada da primeira coluna da linha
    P=0.5 #quantidade de linhas na classe 1 (5) dividido pelo Tamanho da base (10) 5/10 => 0.5

    pp=P

    pr=ss.norm(c0col1M,c0col1std).pdf(col1)

    pp=pp*pr

    pr=ss.norm(c0col2M,c0col2std).pdf(col2)
    pp=pp*pr

    
    # obtendo probabilidade acumulada da segunda coluna da linha
    P=0.5  #quantidade de linhas na classe 2 (5) dividido pelo Tamanho da base (10) 5/10 => 0.5

    pp1=P

    pr=ss.norm(c1col1M,c1col1std).pdf(col1)

    pp1=pp1*pr

    pr=ss.norm(c1col2M,c1col2std).pdf(col2)

    pp1=pp1*pr
    if (pp>pp1):
        m=0
    else:
        m=1
        
    print(" probabilidade classe 0 ", pp , " classe 1 ", pp1, "probMaior ", m)
    

In [97]:
for ind in test.index:
    print(test['class'][ind])
    prob(test['col1'][ind], test['col2'][ind]) 

0
 probabilidade classe 0  0.05032427673372073  classe 1  0.00011557718379945825 probMaior  0
0
 probabilidade classe 0  0.038912298789679556  classe 1  4.02571231326945e-05 probMaior  0
0
 probabilidade classe 0  0.02354195769052507  classe 1  1.0677123311147688e-07 probMaior  0
0
 probabilidade classe 0  0.016404353839568925  classe 1  0.00010893052118437202 probMaior  0
0
 probabilidade classe 0  0.06797274932849016  classe 1  3.892920735657445e-06 probMaior  0
1
 probabilidade classe 0  6.919582110595231e-08  classe 1  0.022027299133895104 probMaior  1
1
 probabilidade classe 0  0.0003624018813567134  classe 1  0.013133373825769767 probMaior  1
1
 probabilidade classe 0  2.4462998433695017e-12  classe 1  0.01893726496422878 probMaior  1
1
 probabilidade classe 0  2.5486090623409466e-08  classe 1  0.041972069978949846 probMaior  1
1
 probabilidade classe 0  1.5425966116475635e-09  classe 1  0.013636752786743601 probMaior  1


In [123]:
### Comparando com modelo Naive Bayes do Scikit Learn

In [113]:
X = test[['col1','col1']] 
y = test[['class']] 

sgd_clf = GaussianNB()

sgd_clf.fit(X, y)
score=sgd_clf.score(X, y)
ypred=sgd_clf.predict(X)
cm = confusion_matrix(y, sgd_clf.predict(X))

# precision, recall, f1 metrics

print("precision:",precision_score(y, ypred))
print("recall:",recall_score(y, ypred))
print("f1:",f1_score(y, ypred))
print("sgd_clf Score: ",score)
print(cm)

precision: 1.0
recall: 1.0
f1: 1.0
sgd_clf Score:  1.0
[[5 0]
 [0 5]]


/home/silvio/.conda/envs/trackml6/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Teste de predição

In [135]:
n=sgd_clf.predict_log_proba(X)
n=sgd_clf.predict(X)

print(np.around(n,3))

[0 0 0 0 0 1 1 1 1 1]


In [137]:
#classe 0
Z=[[2.9873],[1.543540]]

#classe 1
Z=[[7.643521],[0.834567]]
    

n=sgd_clf.predict(Z)
n=sgd_clf.predict_log_proba(Z)

print(np.around(n,4))

[[-34.4051  -0.    ]
 [ -0.     -32.9855]]


In [104]:
print(sgd_clf.class_count_)
print(sgd_clf.class_prior_)
print(sgd_clf.classes_)

print(sgd_clf.sigma_)
print(sgd_clf.theta_)

[5. 5.]
[0.5 0.5]
[0 1]
6.888590843553686e-09
[[0.6868231 0.6868231]
 [1.2190582 1.2190582]]
[[2.7420144  2.7420144 ]
 [7.61465237 7.61465237]]
